In [7]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Euclidean distance function (distance between two points)
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Objective function: Total travel distance for all vehicles
def total_travel_distance(routes, customers, depot):
    total_distance = 0
    for route in routes:
        route_distance = euclidean_distance(depot, customers[route[0]])  # Start from depot
        for i in range(1, len(route)):
            route_distance += euclidean_distance(customers[route[i-1]], customers[route[i]])
        route_distance += euclidean_distance(customers[route[-1]], depot)  # End at depot
        total_distance += route_distance
    return total_distance

# Mutation function to swap two customers in a route (local search)
def mutate(route):
    idx1, idx2 = random.sample(range(len(route)), 2)
    route[idx1], route[idx2] = route[idx2], route[idx1]
    return route

# Parallel Cellular Algorithm for Vehicle Routing Problem (simplified)
def parallel_cellular_algorithm(customers, N_vehicles, T_max, bounds, λ, p_mut):
    N_customers = len(customers)

    # Initialize random routes for each vehicle
    routes = []
    for _ in range(N_vehicles):
        route = random.sample(range(N_customers), N_customers)
        routes.append(route)

    # Set depot location (randomly placed in the middle of the grid for simplicity)
    depot = np.array([bounds[0] + (bounds[1] - bounds[0]) / 2,
                      bounds[0] + (bounds[1] - bounds[0]) / 2])

    best_solution = routes
    best_distance = total_travel_distance(routes, customers, depot)

    # Iterative optimization
    for t in range(T_max):
        new_routes = []

        # Iterate over each vehicle's route (parallelized in theory)
        for vehicle_route in routes:
            # Neighborhood update: consider nearby vehicles
            new_route = vehicle_route.copy()
            for i in range(len(vehicle_route) - 1):
                # Try to swap two customers to reduce travel distance
                new_route = mutate(new_route)

            # Calculate new distance after the update
            new_distance = total_travel_distance([new_route], customers, depot)

            # If the new solution is better, update the route
            if new_distance < best_distance:
                best_solution = [new_route]
                best_distance = new_distance

            # Append the updated route for the current vehicle
            new_routes.append(new_route)

        routes = new_routes  # Update routes for next iteration

        # Reduce output: only print at intervals (e.g., every 100 iterations)
        if t % 100 == 0:
            print(f"Iteration {t+1}: Best total distance = {best_distance}")

    return best_solution, best_distance

# Example usage
N_vehicles = 3  # Number of vehicles
T_max = 1000    # Maximum number of iterations
bounds = (0, 10)  # Grid bounds for customer locations
λ = 0.5         # Diffusion coefficient (not actively used in this example)
p_mut = 0.1     # Mutation probability

# Randomly generate customer locations (excluding the depot)
N_customers = 10
customers = np.random.uniform(bounds[0], bounds[1], size=(N_customers, 2))

# Run the Parallel Cellular Algorithm for routing
best_routes, best_distance = parallel_cellular_algorithm(customers, N_vehicles, T_max, bounds, λ, p_mut)

# Output only final best solution
print(f"\nFinal best routes: {best_routes}")
print(f"Final best total distance: {best_distance}")



Iteration 1: Best total distance = 54.07045371359696
Iteration 101: Best total distance = 41.08999192465061
Iteration 201: Best total distance = 41.08999192465061
Iteration 301: Best total distance = 41.08999192465061
Iteration 401: Best total distance = 39.811071686714016
Iteration 501: Best total distance = 39.811071686714016
Iteration 601: Best total distance = 39.811071686714016
Iteration 701: Best total distance = 36.227804966250595
Iteration 801: Best total distance = 36.227804966250595
Iteration 901: Best total distance = 36.227804966250595

Final best routes: [[8, 3, 7, 6, 0, 4, 9, 5, 1, 2]]
Final best total distance: 36.227804966250595
